In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from copy import copy

In [2]:
path1 = "./data/1/"
path2 = "./data/2/"
path3 = "./data/3/"
path4 = "./data/4/"
path5 = "./data/5/"
path6 = "./data/UpToDate/"

In [3]:
KP = 50
KF = 25

In [4]:
def fix_dollars(apple2):
    new_open = []
    new_close = []
    new_high = []
    new_low = []
    for i in range(len(apple2['Open'])):
        new_open.append(float(apple2['Open'][i][1:]))
        new_close.append(float(apple2['Close/Last'][i][1:]))
        new_high.append(float(apple2['High'][i][1:]))
        new_low.append(float(apple2['Low'][i][1:]))
    

    apple2.drop('Open',axis=1,inplace=True)
    apple2.drop('Close/Last',axis=1,inplace=True)
    apple2.drop('High',axis=1,inplace=True)
    apple2.drop('Low',axis=1,inplace=True)
    

    apple2['Close'] = new_close
    apple2['Open'] = new_open
    apple2['High'] = new_high
    apple2['Low'] = new_low
    
  

In [5]:
def fix_date(apple2):
    month = []
    year = []
    week_start = []
    week_end = []
    flag = False
    for i in range(len(apple2['Date'])):
        month.append(int(apple2['Date'][i][0:2]))
        year.append(int(apple2['Date'][i][6:]))
        today = int(apple2['Date'][i][3:5])
        
        if i == 0:
            tomorrow = int(apple2['Date'][i + 1][3:5])
            if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i + 1][3:5]) - 1 == int(apple2['Date'][i + 2][3:5]) - 2 \
                    == int(apple2['Date'][i + 3][3:5]) - 3 == int(apple2['Date'][i + 4][3:5]) - 4:
                week_start.append(1)
                week_end.append(0)
            elif today + 1 == tomorrow:
                week_start.append(0)
                week_end.append(0)
            else:
                week_start.append(0)
                week_end.append(1)
                flag = True
                continue
        else:
            try:
                tomorrow = int(apple2['Date'][i + 1][3:5])
                if today + 1 == tomorrow:  # I should really be cross referencing with a calendar api, not hacking this together
                    week_start.append(0)
                    week_end.append(0)
                else:
                    week_start.append(0)
                    week_end.append(1)
                    flag = True
                    continue
                if flag:
                    week_start[-1] = 1
                    flag = False
            except:
                yesterday = int(apple2['Date'][i - 1][3:5])
                if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i - 1][3:5]) + 1 == int(
                        apple2['Date'][i - 2][3:5]) + 2 \
                        == int(apple2['Date'][i - 3][3:5]) + 3 == int(apple2['Date'][i - 4][3:5]) + 4:
                    week_start.append(0)
                    week_end.append(1)
                elif yesterday + 1 == today:
                    week_start.append(0)
                    week_end.append(0)
                else:
                    week_start.append(1)
                    week_end.append(0)

    apple2.drop('Date', axis=1, inplace=True)

    apple2['Month'] = month
    apple2['Year'] = year
    apple2['WeekStart'] = week_start
    apple2['WeekEnd'] = week_end


In [6]:
def get_targets(df,KF):
    target_percent_inc = []
    target_best_day = []
    options = ['t--'+str(i)+'Low' for i in range(1,KF+1)]
    for i in range(len(df['t-0Low'])):
        today = df['t-0High'][i]
        best = np.max([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])
        best_day = np.argmax([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])+1
        target_percent_inc.append(round((best-today)*100/today,0))
        target_best_day.append(best_day)
    return target_percent_inc, target_best_day

In [7]:
def fix_shift(apple2,KP,KF,updated=False):
    #KP = 50
    #changing KP and KF from 10 to 30 causes accuracy to drop...Why? Possibly because there are now more classes for y
    #KF = 25

    names = ['Volume','Close','Open','High','Low','Month','Year'] 
    cases = pd.DataFrame(apple2.drop('WeekStart',axis=1).drop('WeekEnd',axis=1).values)
    df = pd.concat([cases.shift(i) for i in range(KP-int(updated),-KF-1,-1)],axis=1).join(apple2['WeekStart']).join(apple2['WeekEnd'])
    df.columns = ['t-'+str(i+int(updated))+x for i in range(KP-int(updated),-KF-1,-1) for x in names]+['WeekStart','WeekEnd']
    return df

In [8]:
def fix_normalize(apple2,df):
    for apple_col in list(apple2):
        if apple_col in ['WeekStart','WeekEnd']:
            continue
        MEAN = apple2[apple_col].mean()
        STD = apple2[apple_col].std()
        for df_col in list(df):
            if apple_col in df_col:
                if apple_col =='Month' or apple_col == 'Year':
                    df = df.astype({df_col: 'int32'})
                else:
                    df[df_col]=round((df[df_col]-MEAN)/STD,0)

In [17]:
def drop_future(df,KF):
    for i in range(0,-KF-1,-1):
        for x in ['Month','Year','Volume','Close','Open','High','Low']:
            df.drop('t-'+str(i)+x,axis=1,inplace=True)

In [9]:
def fix_reverse(apple2):
    return apple2.loc[::-1].reset_index(drop=True)

In [10]:
apple = pd.read_csv(path5+"aapl_historical.csv")
apple2 = apple.copy(deep=True)
fix_dollars(apple2)
apple2 = fix_reverse(apple2)
apple2

,Date,Volume,Close,Open,High,Low
0,05/16/2013,602950359,15.5206,15.1157,15.6375,14.9607
1,05/17/2013,427864749,15.4736,15.6804,15.7175,15.3932
2,05/20/2013,450118020,15.8189,15.4254,15.9214,15.3607
3,05/21/2013,455670978,15.7021,15.6482,15.9100,15.5071
4,05/22/2013,442730783,15.7626,15.8589,16.0125,15.6507
...,...,...,...,...,...,...
2512,05/09/2023,45326870,171.7700,173.0500,173.5400,171.6000
2513,05/10/2023,53724500,173.5550,173.0200,174.0300,171.9000
2514,05/11/2023,49514680,173.7500,173.8500,174.5900,172.1700
2515,05/12/2023,45533140,172.5700,173.6200,174.0600,171.0000


In [11]:
fix_date(apple2)
apple2

,Volume,Close,Open,High,Low,Month,Year,WeekStart,WeekEnd
0,602950359,15.5206,15.1157,15.6375,14.9607,5,2013,0,0
1,427864749,15.4736,15.6804,15.7175,15.3932,5,2013,0,1
2,450118020,15.8189,15.4254,15.9214,15.3607,5,2013,1,0
3,455670978,15.7021,15.6482,15.9100,15.5071,5,2013,0,0
4,442730783,15.7626,15.8589,16.0125,15.6507,5,2013,0,0
...,...,...,...,...,...,...,...,...,...
2512,45326870,171.7700,173.0500,173.5400,171.6000,5,2023,0,0
2513,53724500,173.5550,173.0200,174.0300,171.9000,5,2023,0,0
2514,49514680,173.7500,173.8500,174.5900,172.1700,5,2023,0,0
2515,45533140,172.5700,173.6200,174.0600,171.0000,5,2023,0,1


In [12]:
df = fix_shift(apple2,KP,KF)
df

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2013.0,480849968.0,14.7679,14.9461,15.0000,14.5750,6.0,2013.0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2013.0,480187768.0,14.3764,14.5500,14.5950,14.2161,6.0,2013.0,0,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2013.0,314132994.0,14.3796,14.4893,14.5639,14.2439,6.0,2013.0,1,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2013.0,367647133.0,14.2168,14.4250,14.4568,14.1307,6.0,2013.0,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2013.0,336950745.0,14.0636,14.2589,14.3354,14.0550,6.0,2013.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,44998470.0,147.92,147.710,149.1700,147.45,2.0,2023.0,50547000.0,147.41,147.050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2513,50547000.0,147.41,147.050,149.0800,146.83,2.0,2023.0,55478990.0,145.31,146.830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2514,55478990.0,145.31,146.830,147.2285,145.01,3.0,2023.0,52279760.0,145.91,144.380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2515,52279760.0,145.91,144.380,146.7100,143.90,3.0,2023.0,70732300.0,151.03,148.045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [13]:
def fix_null(df):
    df.dropna(inplace=True)
    return df.reset_index(drop=True)
    

In [14]:
df = fix_null(df)
df

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,602950359.0,15.5206,15.1157,15.6375,14.9607,5.0,2013.0,427864749.0,15.4736,15.6804,...,2013.0,331867627.0,17.4493,17.6107,17.8786,17.4054,9.0,2013.0,1,0
1,427864749.0,15.4736,15.6804,15.7175,15.3932,5.0,2013.0,450118020.0,15.8189,15.4254,...,2013.0,344063862.0,17.8104,17.8414,17.9372,17.7243,9.0,2013.0,0,0
2,450118020.0,15.8189,15.4254,15.9214,15.3607,5.0,2013.0,455670978.0,15.7021,15.6482,...,2013.0,235116802.0,17.6882,17.8661,17.8814,17.6300,9.0,2013.0,0,1
3,455670978.0,15.7021,15.6482,15.9100,15.5071,5.0,2013.0,442730783.0,15.7626,15.8589,...,2013.0,357928337.0,17.7936,17.8014,17.8350,17.4982,9.0,2013.0,1,0
4,442730783.0,15.7626,15.8589,16.0125,15.6507,5.0,2013.0,352959179.0,15.7907,15.5696,...,2013.0,337917025.0,18.0775,18.0357,18.1400,17.9814,9.0,2013.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,80223630.0,137.8700,135.2800,138.0200,134.2200,1.0,2023.0,81760310.0,141.1100,138.1200,...,2023.0,45326870.0,171.7700,173.0500,173.5400,171.6000,5.0,2023.0,1,0
2438,81760310.0,141.1100,138.1200,143.3150,137.9000,1.0,2023.0,66435140.0,142.5300,140.3050,...,2023.0,53724500.0,173.5550,173.0200,174.0300,171.9000,5.0,2023.0,0,0
2439,66435140.0,142.5300,140.3050,143.1600,140.3000,1.0,2023.0,65799350.0,141.8600,140.8900,...,2023.0,49514680.0,173.7500,173.8500,174.5900,172.1700,5.0,2023.0,0,0
2440,65799350.0,141.8600,140.8900,142.4300,138.8100,1.0,2023.0,54105070.0,143.9600,143.1700,...,2023.0,45533140.0,172.5700,173.6200,174.0600,171.0000,5.0,2023.0,0,1


In [15]:
target_percent_inc, target_best_day = get_targets(df,KF)
np.average(target_percent_inc)

4.868140868140868

In [16]:
fix_normalize(apple2,df)
df

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t--24Year,t--25Volume,t--25Close,t--25Open,t--25High,t--25Low,t--25Month,t--25Year,WeekStart,WeekEnd
0,4.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,2.0,-1.0,-1.0,-1.0,-1.0,9.0,2013.0,1,0
1,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,2.0,-1.0,-1.0,-1.0,-1.0,9.0,2013.0,0,0
2,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,1.0,-1.0,-1.0,-1.0,-1.0,9.0,2013.0,0,1
3,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,2.0,-1.0,-1.0,-1.0,-1.0,9.0,2013.0,1,0
4,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,2.0,-1.0,-1.0,...,2013.0,2.0,-1.0,-1.0,-1.0,-1.0,9.0,2013.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,1.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,1,0
2438,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,1.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,0,0
2439,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,1.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,0,0
2440,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,2.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,0,1


In [18]:
drop_future(df,KF)
df

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,4.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,0.0,-1.0,-1.0,-1.0,-1.0,7.0,2013.0,1,0
1,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,1.0,-1.0,-1.0,-1.0,-1.0,7.0,2013.0,0,0
2,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,2.0,-1.0,-1.0,-1.0,-1.0,7.0,2013.0,0,1
3,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,3.0,-1.0,-1.0,...,2013.0,2.0,-1.0,-1.0,-1.0,-1.0,7.0,2013.0,1,0
4,3.0,-1.0,-1.0,-1.0,-1.0,5.0,2013.0,2.0,-1.0,-1.0,...,2013.0,1.0,-1.0,-1.0,-1.0,-1.0,8.0,2013.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2437,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,1.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,3.0,2023.0,1,0
2438,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,1.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,4.0,2023.0,0,0
2439,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,1.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,4.0,2023.0,0,0
2440,-1.0,1.0,1.0,1.0,1.0,1.0,2023.0,-1.0,2.0,1.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,4.0,2023.0,0,1


In [19]:
#For better accuracy
def round_target(y,floor):
    for i in range(len(y)):
        curr = y[i]
        if 0<curr<floor:
            y[i] = 1
        elif -floor<curr<0:
            y[i] = -1
        else:
            y[i] = curr//floor
            if y[i] > 3:
                y[i] = 3
            if y[i] < -3:
                y[i] = -3

In [20]:
#finds min percent gains over KF days to double money in 1 year
def double_yearly(KF):
    TRADING_DAYS = 252
    periods = TRADING_DAYS/KF
    perc = (-1+2**(1/periods))*100
    return perc

In [21]:
def round_day(y):
    for i in range(len(y)):
        y[i] = y[i]//5

In [22]:
X = df
y = copy(target_percent_inc)
y_day = copy(target_best_day)
round_day(y_day)
round_target(y,double_yearly(KF))
y

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 -0.0,
 0.0,
 0.0,
 -1,
 -1,
 -1,
 -0.0,
 0.0,
 -1,
 0.0,
 1,
 1,
 1,
 1,
 0.0,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1,
 1,
 1,
 1.0,
 1,
 1,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 0.0,
 0.0,
 -0.0,
 -1,
 -0.0,
 -1,
 -0.0,
 -1,
 0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -0.0,
 -1,
 -1,
 -1,
 -1,
 1,
 1.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.0,
 -0.0,
 -1,
 -1,
 0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.0,
 1,
 0.0,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0,
 1.0,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1,
 1,
 1,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression

In [66]:
clf = MLPClassifier(random_state=0)
clf_day = MLPClassifier(random_state=1)

In [63]:
scores = cross_val_score(clf, X, y, cv=3) #noticable improvement in variance and aveerage after adding week start and end
#except for the fact that without random_state=0 it can still be pretty bad...
#I know this is really bad practice, but for now I don't care
scores

array([0.70515971, 0.70515971, 0.21130221])

In [50]:
scores_day = cross_val_score(clf_day, X, y_day, cv=3)#not really any better
scores_day

array([0.20884521, 0.28501229, 0.18181818])

In [67]:
clf.fit(X,y)
clf_day.fit(X,y_day)

MLPClassifier(random_state=1)

In [52]:
uptodate = pd.read_csv(path6+"apple.csv")
uptodate

,Date,Close/Last,Volume,Open,High,Low
0,05/19/2023,$175.16,55809480,$176.39,$176.39,$174.94
1,05/18/2023,$175.05,65496660,$173.00,$175.24,$172.58
2,05/17/2023,$172.69,57951600,$171.71,$172.925,$170.4201
3,05/16/2023,$172.07,42110290,$171.99,$173.1383,$171.7991
4,05/15/2023,$172.07,37266660,$173.16,$173.21,$171.47
...,...,...,...,...,...,...
119,11/28/2022,$144.22,69346520,$145.14,$146.64,$143.38
120,11/25/2022,$148.11,35195860,$148.305,$148.88,$147.12
121,11/23/2022,$151.07,58301400,$149.45,$151.83,$149.34
122,11/22/2022,$150.18,51804130,$148.13,$150.42,$146.925


In [53]:
u2 = uptodate.copy(deep=True)
fix_dollars(u2)
u2 = u2.loc[::-1].reset_index(drop=True)
u2

,Date,Volume,Close,Open,High,Low
0,11/21/2022,58724070,148.01,150.160,150.3700,147.7150
1,11/22/2022,51804130,150.18,148.130,150.4200,146.9250
2,11/23/2022,58301400,151.07,149.450,151.8300,149.3400
3,11/25/2022,35195860,148.11,148.305,148.8800,147.1200
4,11/28/2022,69346520,144.22,145.140,146.6400,143.3800
...,...,...,...,...,...,...
119,05/15/2023,37266660,172.07,173.160,173.2100,171.4700
120,05/16/2023,42110290,172.07,171.990,173.1383,171.7991
121,05/17/2023,57951600,172.69,171.710,172.9250,170.4201
122,05/18/2023,65496660,175.05,173.000,175.2400,172.5800


In [54]:
fix_date(u2)

In [55]:
u2

,Volume,Close,Open,High,Low,Month,Year,WeekStart,WeekEnd
0,58724070,148.01,150.160,150.3700,147.7150,11,2022,0,0
1,51804130,150.18,148.130,150.4200,146.9250,11,2022,0,0
2,58301400,151.07,149.450,151.8300,149.3400,11,2022,0,1
3,35195860,148.11,148.305,148.8800,147.1200,11,2022,0,1
4,69346520,144.22,145.140,146.6400,143.3800,11,2022,1,0
...,...,...,...,...,...,...,...,...,...
119,37266660,172.07,173.160,173.2100,171.4700,5,2023,1,0
120,42110290,172.07,171.990,173.1383,171.7991,5,2023,0,0
121,57951600,172.69,171.710,172.9250,170.4201,5,2023,0,0
122,65496660,175.05,173.000,175.2400,172.5800,5,2023,0,0


In [56]:
dfu = fix_shift(u2,50,0,updated=True)
dfu

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,58724070.0,148.01,150.160,150.3700,147.7150,11.0,2022.0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,51804130.0,150.18,148.130,150.4200,146.9250,11.0,2022.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,58301400.0,151.07,149.450,151.8300,149.3400,11.0,2022.0,0,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,35195860.0,148.11,148.305,148.8800,147.1200,11.0,2022.0,0,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022.0,69346520.0,144.22,145.140,146.6400,143.3800,11.0,2022.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,87558030.0,153.83,153.785,156.3000,153.4600,3.0,2023.0,56182030.0,151.60,153.700,...,2023.0,37266660.0,172.07,173.160,173.2100,171.4700,5.0,2023.0,1,0
120,56182030.0,151.60,153.700,154.0299,151.1300,3.0,2023.0,47204790.0,152.87,152.810,...,2023.0,42110290.0,172.07,171.990,173.1383,171.7991,5.0,2023.0,0,0
121,47204790.0,152.87,152.810,153.4700,151.8300,3.0,2023.0,53833580.0,150.59,153.559,...,2023.0,57951600.0,172.69,171.710,172.9250,170.4201,5.0,2023.0,0,0
122,53833580.0,150.59,153.559,154.5350,150.2250,3.0,2023.0,68572400.0,148.50,150.210,...,2023.0,65496660.0,175.05,173.000,175.2400,172.5800,5.0,2023.0,0,0


In [57]:
dfu.dropna(inplace=True)
dfu = dfu.reset_index(drop=True)
dfu

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,58724070.0,148.01,150.160,150.3700,147.7150,11.0,2022.0,51804130.0,150.18,148.130,...,2023.0,118339000.0,150.82,148.900,151.1800,148.1700,2.0,2023.0,0,0
1,51804130.0,150.18,148.130,150.4200,146.9250,11.0,2022.0,58301400.0,151.07,149.450,...,2023.0,154357300.0,154.50,148.030,157.3800,147.8300,2.0,2023.0,0,1
2,58301400.0,151.07,149.450,151.8300,149.3400,11.0,2022.0,35195860.0,148.11,148.305,...,2023.0,69858310.0,151.73,152.575,153.1000,150.7800,2.0,2023.0,1,0
3,35195860.0,148.11,148.305,148.8800,147.1200,11.0,2022.0,69346520.0,144.22,145.140,...,2023.0,83322550.0,154.65,150.640,155.2300,150.6400,2.0,2023.0,0,0
4,69346520.0,144.22,145.140,146.6400,143.3800,11.0,2022.0,83763800.0,141.17,144.290,...,2023.0,64120080.0,151.92,153.880,154.5800,151.1680,2.0,2023.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,87558030.0,153.83,153.785,156.3000,153.4600,3.0,2023.0,56182030.0,151.60,153.700,...,2023.0,37266660.0,172.07,173.160,173.2100,171.4700,5.0,2023.0,1,0
71,56182030.0,151.60,153.700,154.0299,151.1300,3.0,2023.0,47204790.0,152.87,152.810,...,2023.0,42110290.0,172.07,171.990,173.1383,171.7991,5.0,2023.0,0,0
72,47204790.0,152.87,152.810,153.4700,151.8300,3.0,2023.0,53833580.0,150.59,153.559,...,2023.0,57951600.0,172.69,171.710,172.9250,170.4201,5.0,2023.0,0,0
73,53833580.0,150.59,153.559,154.5350,150.2250,3.0,2023.0,68572400.0,148.50,150.210,...,2023.0,65496660.0,175.05,173.000,175.2400,172.5800,5.0,2023.0,0,0


In [58]:
fix_normalize(apple2,dfu)
dfu

,t-50Volume,t-50Close,t-50Open,t-50High,t-50Low,t-50Month,t-50Year,t-49Volume,t-49Close,t-49Open,...,t-2Year,t-1Volume,t-1Close,t-1Open,t-1High,t-1Low,t-1Month,t-1Year,WeekStart,WeekEnd
0,-1.0,2.0,2.0,2.0,2.0,11.0,2022.0,-1.0,2.0,2.0,...,2023.0,-0.0,2.0,2.0,2.0,2.0,2.0,2023.0,0,0
1,-1.0,2.0,2.0,2.0,2.0,11.0,2022.0,-1.0,2.0,2.0,...,2023.0,-0.0,2.0,2.0,2.0,2.0,2.0,2023.0,0,1
2,-1.0,2.0,2.0,2.0,2.0,11.0,2022.0,-1.0,2.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,2.0,2023.0,1,0
3,-1.0,2.0,2.0,2.0,2.0,11.0,2022.0,-1.0,2.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,2.0,2023.0,0,0
4,-1.0,2.0,2.0,2.0,2.0,11.0,2022.0,-1.0,1.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,2.0,2023.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,-1.0,2.0,2.0,2.0,2.0,3.0,2023.0,-1.0,2.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,1,0
71,-1.0,2.0,2.0,2.0,2.0,3.0,2023.0,-1.0,2.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,0,0
72,-1.0,2.0,2.0,2.0,2.0,3.0,2023.0,-1.0,2.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,0,0
73,-1.0,2.0,2.0,2.0,2.0,3.0,2023.0,-1.0,2.0,2.0,...,2023.0,-1.0,2.0,2.0,2.0,2.0,5.0,2023.0,0,0


In [59]:
point = np.array(dfu.iloc[-1]).reshape(1,-1)
point

array([[-1.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,
         3.000e+00,  2.023e+03, -1.000e+00,  2.000e+00,  2.000e+00,
         2.000e+00,  2.000e+00,  3.000e+00,  2.023e+03, -1.000e+00,
         2.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,  3.000e+00,
         2.023e+03, -1.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,
         2.000e+00,  3.000e+00,  2.023e+03, -1.000e+00,  2.000e+00,
         2.000e+00,  2.000e+00,  2.000e+00,  3.000e+00,  2.023e+03,
        -1.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,
         3.000e+00,  2.023e+03, -1.000e+00,  2.000e+00,  2.000e+00,
         2.000e+00,  2.000e+00,  3.000e+00,  2.023e+03, -1.000e+00,
         2.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,  3.000e+00,
         2.023e+03, -1.000e+00,  2.000e+00,  2.000e+00,  2.000e+00,
         2.000e+00,  3.000e+00,  2.023e+03, -1.000e+00,  2.000e+00,
         2.000e+00,  2.000e+00,  2.000e+00,  3.000e+00,  2.023e+03,
        -1.000e+00,  2.000e+00,  2.000e+00,  2.0

In [68]:
clf.predict(point)#Now it's saying apple will go down.
#With random state = 0 and 3 cross val scores approx 70%, predict apple will go up by less than ~8%
#With random state = 1 and 2 good scores 1 bad, predict apple will go down by less than 8% (given tomorrow's high,)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([-1.])

In [61]:
clf_day.predict(point)

C:\Users\phill\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


array([2], dtype=int64)